In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../ai_medreview/data/data.csv')
data.shape

(8086, 19)

In [3]:
from ai_medreview.data import *

/Users/janduplessis/.pyenv/versions/3.12.1/envs/lewagon/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

In [4]:
df = load_google_sheet()

2024-06-24 14:04:10.067 | INFO     | ai_medreview.utils:wrapper:19 - 🖥️    Started: 'load_google_sheet'
2024-06-24 14:04:14.262 | INFO     | ai_medreview.utils:wrapper:25 - ✅ Completed: 'load_google_sheet' ⚡️4.193807 sec


In [5]:
df.head()

,submission_id,respondent-id,time,rating,free_text,do_better,pcn,surgery
5386,mannual,upload,2022-12-14 15:36:00,Extremely likely,Nothing,,Brompton-Health-PCN,Scarsdale-Medical-Centre
5387,mannual,upload,2022-12-14 15:36:00,Extremely likely,Really Likely! Extremely likely Practice!!!!,,Brompton-Health-PCN,Scarsdale-Medical-Centre
5388,mannual,upload,2022-12-15 16:43:00,Extremely likely,"Appointments sometimes are far ahead, but over...",,Brompton-Health-PCN,Scarsdale-Medical-Centre
5389,mannual,upload,2022-12-15 18:15:00,Extremely likely,Nothing I can think of,,Brompton-Health-PCN,Scarsdale-Medical-Centre
5390,mannual,upload,2022-12-16 11:21:00,Extremely likely,Very helpful and prompt service. Definitely re...,,Brompton-Health-PCN,Scarsdale-Medical-Centre


In [6]:
def find_people(text):

    # Check if the text is empty or not a string
    if not text or not isinstance(text, str):
        return text  # Return the text as-is if it's invalid or empty

    people = []
    try:
        # Run the NER pipeline on the valid input text
        entities = ner_pipeline(text)

        # Iterate over detected entities
        for entity in entities:
            # Check if the entity is classified as a person
            if entity["entity_group"] == "PER":
                # Add the persons name to a list.
               people.append(entity["word"])
    except ValueError as e:
        # Log the error for debugging
        print(f"Error processing text: {text}")
        raise e

    return people

In [7]:
df["free_text_people"] = df["free_text"].progress_apply(find_people)

100%|█████████████████████████████████████████████████████████████████████████████████| 8087/8087 [25:37<00:00,  5.26it/s]


In [ ]:
people1 = df2['free_text_people'].to_list()

In [ ]:
for l in df2['free_text_people']:
    if len(list(l)) != 0:
        print(list(l))

In [ ]:
df["do_better_people"] = df["do_better"].progress_apply(find_people)

In [ ]:
for l in df['do_better_people']:
    if len(list(l)) != 0:
        print(list(l))

In [ ]:
people2 = df["do_better_people"].to_list()

In [ ]:
import pandas as pd
from transformers import pipeline

# Initialize the NER pipeline
ner_pipeline = pipeline("ner", grouped_entities=True)

def find_people(text):
    """
    Extract names of people from the given text using NER.
    
    Args:
    text (str): The input text to process.
    
    Returns:
    list: A list of names classified as persons.
    """
    if not text or not isinstance(text, str):
        return []  # Return an empty list if text is invalid or empty

    people = []
    try:
        entities = ner_pipeline(text)
        for entity in entities:
            if entity["entity_group"] == "PER":
                people.append(entity["word"])
    except ValueError as e:
        print(f"Error processing text: {text}")
        raise e

    return people

def extract_people_from_dataframe(df, text_column):
    """
    Process each row in the DataFrame to extract names of people from a specified text column.
    
    Args:
    df (pd.DataFrame): The input DataFrame containing text data.
    text_column (str): The name of the column containing text data.
    
    Returns:
    pd.DataFrame: A new DataFrame with an additional column for extracted names.
    """
    if text_column not in df.columns:
        raise ValueError(f"Column '{text_column}' does not exist in the DataFrame")

    # Apply the find_people function to each row in the specified text column
    df['people'] = df[text_column].apply(find_people)
    
    return df

# Example usage
if __name__ == "__main__":
    # Sample DataFrame
    data = {
        'id': [1, 2, 3],
        'text': [
            "Barack Obama was the 44th President of the United States.",
            "Elon Musk is the CEO of SpaceX and Tesla.",
            "Serena Williams is a professional tennis player and so is Stephie Graff, and Sammy David Junior"
        ]
    }
    df = pd.DataFrame(data)
    
    # Extract names of people from the text column
    result_df = extract_people_from_dataframe(df, 'text')
    
    # Display the result
    print(result_df)